In [ ]:
!pip install gpustat
!pip install --upgrade keras
!pip install --upgrade keras_preprocessing
!pip install segmentation_models
!pip install imgaug
!gpustat

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import segmentation_models as sm
import imgaug as ia
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
data_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_data/train/'
label_dir = '/content/gdrive/My Drive/data_source/food_segmentation/unet/training_label/train/'

In [ ]:
BATCH_SIZE = 4
GENERATE_SEED = np.random.randint(0, 1024)
VALIDATION_SPLIT = 0.2

maskgen_args = dict(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=VALIDATION_SPLIT
)

def image_preprocessing(images):
    sometimes = lambda x: ia.augmenters.Sometimes(0.5, x)
    augmentation_sequence = ia.augmenters.Sequential([
        sometimes(ia.augmenters.AdditiveGaussianNoise(loc=(0.0, 0.1))),
        sometimes(ia.augmenters.Add(-0.1, 0.1))
    ])
    return augmentation_sequence(images=images)


imagegen_args = dict(
    samplewise_center=True,
    samplewise_std_normalization=True,
    preprocessing_function=image_preprocessing,
    **maskgen_args
)

image_datagen = keras.preprocessing.image.ImageDataGenerator(**imagegen_args)
mask_datagen = keras.preprocessing.image.ImageDataGenerator(**maskgen_args)

training_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='training'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='training'
    )
)

validation_data_generator = zip(
    image_datagen.flow_from_directory(
        data_dir,
        target_size=(512, 512),
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='jpeg',
        seed=GENERATE_SEED,
        subset='validation'
    ),
    mask_datagen.flow_from_directory(
        label_dir,
        target_size=(512, 512),
        color_mode='grayscale',
        batch_size=BATCH_SIZE,
        class_mode=None,
        save_format='png',
        seed=GENERATE_SEED,
        subset='validation'
    )
)

In [ ]:
# Lovasz Softmax
# Reference: https://github.com/bermanmaxim/LovaszSoftmax

def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.math.cumsum(gt_sorted)
    union = gts + tf.math.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------
def lovasz_hinge(labels, logits, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

In [ ]:
unet_model = sm.Unet(backbone_name='inceptionresnetv2', encoder_freeze=True, classes=1, activation='linear')
unet_model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    loss=lovasz_hinge,
    metrics=['accuracy']
)

In [ ]:
unet_model.fit_generator(
    training_data_generator,
    epochs=6,
    steps_per_epoch=2048,
    callbacks=[keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/data_source/food_segmentation/sm_unet/model_checkpoint_{epoch:02d}.hdf5")]
)

In [ ]:
unet_model.load_weights('/content/gdrive/My Drive/data_source/food_segmentation/sm_unet/6_epochs.hdf5')

In [ ]:
unet_model.fit_generator(
    training_data_generator,
    epochs=6,
    steps_per_epoch=2048,
    callbacks=[keras.callbacks.ModelCheckpoint("/content/gdrive/My Drive/data_source/food_segmentation/sm_unet/model_checkpoint_{epoch:02d}_2.hdf5")]
)